In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install fastseg --q
!pip install torchinfo --q
!pip install albumentations --q
!pip install geffnet --q
!pip install tqdm

In [ ]:
import fastseg
from fastseg import MobileV3Large
from PIL import Image
import cv2 as cv
import albumentations as A
import tqdm
from torchinfo import summary
import matplotlib.pyplot as plt
import torch
from torchvision import transforms
import matplotlib.pyplot as plt
from albumentations.pytorch import ToTensorV2
import numpy as np
import torch
import torch.nn as nn
import os
import shutil
import random
from torch.utils.data import Dataset , DataLoader
import torchvision

In [ ]:
import os
import shutil

# Define the directories
jpeg_folder = '/kaggle/working/jpeg_images'  # For JPEG images
png_folder = '/kaggle/working/png_masks'      # For PNG masks

# Remove the directories if they exist
if os.path.exists(jpeg_folder):
    shutil.rmtree(jpeg_folder)  # Remove the jpeg_images directory and its contents
if os.path.exists(png_folder):
    shutil.rmtree(png_folder)  # Remove the png_masks directory and its contents

# Now create new directories
os.makedirs(jpeg_folder, exist_ok=True)
os.makedirs(png_folder, exist_ok=True)

# Define the original folder for copying files
original_folder = '/kaggle/input/train3/1'  # Replace with your actual path

# Get a sorted list of all files in the original folder
files = sorted(os.listdir(original_folder))

# Initialize a counter for numbering
counter = 1

# Copy files in pairs (image and mask)
for i in range(0, len(files), 2):
    # Get the image file
    image_file = files[i]
    image_path = os.path.join(original_folder, image_file)
    
    # Check if there's a corresponding mask file
    if i + 1 < len(files):
        mask_file = files[i + 1]
        mask_path = os.path.join(original_folder, mask_file)
        
        # Copy the unannotated image to the JPEG folder if it's a JPEG file
        if image_file.lower().endswith('.jpeg') or image_file.lower().endswith('.jpg'):
            new_image_name = f"image_{counter}.jpeg"  # Create a new name with numbering
            shutil.copy(image_path, os.path.join(jpeg_folder, new_image_name))
        
        # Copy the annotated mask to the PNG folder if it's a PNG file
        if mask_file.lower().endswith('.png'):
            new_mask_name = f"mask_{counter}.png"  # Create a new name with numbering
            shutil.copy(mask_path, os.path.join(png_folder, new_mask_name))
        else:
            print(f"Warning: Mask file '{mask_file}' is not a PNG file.")
    else:
        print(f"Warning: No mask found for image '{image_file}'.")
    
    # Increment the counter after processing each pair
    counter += 1

print("Files have been successfully copied and renamed.")


In [ ]:
model = MobileV3Large.from_pretrained()

In [ ]:
model

In [ ]:
model.conv_up3

In [ ]:
model.conv_up2


In [ ]:


# Add segmentation-specific layers (last and second-last layers)
model.second_last = nn.Sequential(
    nn.Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1)),
    nn.ReLU(),
    nn.BatchNorm2d(128),
    nn.Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1)),
    nn.ReLU(),
    nn.BatchNorm2d(64)
)

model.last = nn.Sequential(
    nn.Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1)),
    nn.ReLU(),
    nn.BatchNorm2d(128),
    nn.Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1)),
    nn.ReLU(),
    nn.BatchNorm2d(64),
    nn.Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1)),
    nn.ReLU(),
    nn.BatchNorm2d(32),
    nn.Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1)),
    nn.ReLU(),
    nn.BatchNorm2d(16),
    nn.Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1)),
    nn.ReLU(),
    nn.BatchNorm2d(8),
    nn.Conv2d(8, 4, kernel_size=(1, 1), stride=(1, 1)),
    nn.ReLU(),
    nn.BatchNorm2d(4),
    nn.Conv2d(4, 1, kernel_size=(1, 1), stride=(1, 1))
)


In [ ]:
model

In [ ]:
# Freeze all layers in the model
for param in model.parameters():
    param.requires_grad = False


In [ ]:
# Count total trainable parameters
def count_trainable_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

trainable_params = count_trainable_params(model)
print(f"Total Trainable Parameters: {trainable_params}")


In [ ]:
# Unfreeze the last layer
for param in model.last.parameters():
    param.requires_grad = True

# Unfreeze the second-last layer
for param in model.second_last.parameters():
    param.requires_grad = True


In [ ]:
# Count total trainable parameters
def count_trainable_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

trainable_params = count_trainable_params(model)
print(f"Total Trainable Parameters: {trainable_params}")


In [ ]:
# Unfreeze the conv_up2 layer
for param in model.conv_up2.parameters():
    param.requires_grad = True

# Unfreeze the conv_up3 layer
for param in model.conv_up3.parameters():
    param.requires_grad = True


In [ ]:
# Count total trainable parameters
def count_trainable_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

trainable_params = count_trainable_params(model)
print(f"Total Trainable Parameters: {trainable_params}")


In [ ]:
import torch

# Assuming 'model' is your MobileNetV3 model
# Accessing block4 from the model
block4 = model.trunk.block4

# Unfreeze all layers in block4
for module in block4:
    for param in module.parameters():
        param.requires_grad = True

# Check if the parameters in block4 are unfrozen
for module in block4:
    for param in module.parameters():
        print(f'Parameter {param} requires grad: {param.requires_grad}')


In [ ]:
# Count total trainable parameters
def count_trainable_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

trainable_params = count_trainable_params(model)
print(f"Total Trainable Parameters: {trainable_params}")


In [ ]:
device = 'cuda'

In [ ]:
model = model.to('cuda')

In [ ]:
class Dataset_Builder(Dataset):
    def __init__(self,img_list ,mask_list ):
        super().__init__()
        self.img_list = img_list
        self.mask_list = mask_list
        self.augmentation_transform = A.Compose([
#     A.HorizontalFlip(p=0.5),  # Randomly flip images horizontally
    A.Rotate(limit=(-180, 180) , p=0.5 ),
#     A.Perspective(scale=(0.05, 0.1), p=0.5)
#     A.RandomResizedCrop(height=512, width=512, scale=(0.8, 1.0), p=0.5),

        ])
        
        self.augmentation_transform_img = A.Compose([
                                A.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.2, p=0.5),  # Vary color conditions
                                A.GaussianBlur(blur_limit=(3, 5), p=0.3),  # Simulate Gaussian noise
                                A.MotionBlur(blur_limit=7, p=0.4),  # Apply motion blur with a probability of 40%
                                A.RandomSunFlare(flare_roi=(0.3, 0.3, 1, 0.6), angle_lower=0.5, src_radius=100, p=0.3),
                                A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], p=1.0),
                                ToTensorV2()])
        
        self.augmentation_transform_mask = A.Compose([
            ToTensorV2()
        ])
        
    def __len__(self):
        return len(self.img_list)
    
    def __getitem__(self,idx):
        mask = cv.imread('/kaggle/working/png_masks/'+ self.mask_list[idx])
        img = cv.imread('/kaggle/working/jpeg_images/'+self.img_list[idx])
       
        
        img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
        mask = cv.cvtColor(mask, cv.COLOR_BGR2RGB)
#         print(mask)
        img = cv.resize(img, (512, 512))
        mask = cv.resize(mask, (512, 512))
        augmented = self.augmentation_transform(image=img, mask=mask)
        img = augmented['image']
        mask = augmented['mask']
        img = self.augmentation_transform_img(image=img)['image']
        mask = self.augmentation_transform_mask(image=mask)['image']

        return img,mask

IMG_DIR = "/kaggle/working/jpeg_images"
MASK_DIR = "/kaggle/working/png_masks"

imagedir = os.listdir(IMG_DIR)
maskdir = os.listdir(MASK_DIR)

imagedir.sort()
maskdir.sort()

# Create dataset and dataloaders
dataset = Dataset_Builder(img_list=imagedir, mask_list=maskdir)
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# Dataloaders for training and validation
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

print("Dataloaders are ready for training and validation.")


In [ ]:
import matplotlib.pyplot as plt

# Assuming train_dataset[0] returns (image, mask)
image, mask = train_dataset[1]  # Replace with your dataset access method

# Plotting the image
plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.imshow(image.permute(1, 2, 0))  # If using PyTorch, permute to (H, W, C)
plt.title('Image')
plt.axis('off')

# Plotting the mask
plt.subplot(1, 2, 2)
plt.imshow(mask.permute(1, 2, 0), cmap='gray')  # Use gray colormap for mask
plt.title('Mask')
plt.axis('off')

plt.show()


In [ ]:
device = 'cuda'
num_epochs = 25
optimizer = torch.optim.AdamW(params=model.parameters())
criterion = torch.nn.BCEWithLogitsLoss()


In [ ]:
summary(model)

In [ ]:
def IOU(pred_image, ground_truth):
    total_iou = 0
    batch_size = pred_image.shape[0]

    for pred, ground in zip(pred_image, ground_truth):
        pred = pred.flatten()
        ground = ground.flatten()
        
        # Calculate intersection
        intersection = np.sum((pred == 1) & (ground == 1)).astype(int)
        
        # Calculate union
        union = np.sum((pred == 1) | (ground == 1)).astype(int)
        
        # Calculate IoU
        if union == 0:
            # If both are empty, define IoU as 1 (perfect match)
            iou = 1
        else:
            iou = intersection / union
        
        total_iou += iou  # Accumulate IoU for the current batch

    return total_iou / batch_size if batch_size > 0 else 0  # Return the average IoU

In [ ]:
import torch
from tqdm import tqdm  # Import tqdm for the progress bar


save_path = "saved_models/"

os.makedirs(save_path, exist_ok=True)
val_epoch_loss_list = []
val_batch_loss_list = []
train_epoch_loss_list = []
train_batch_loss_list = []
IOU_Score_list = []

for epoch in range(num_epochs):
    
    train_epoch_loss = 0
    val_epoch_loss = 0
    batch = 0

    # Wrap the DataLoader with tqdm for a progress bar
    with tqdm(train_loader, unit="batch") as tepoch:
        tepoch.set_description(f"Epoch {epoch+1}/{num_epochs}")
        model.train()
        for img, mask in tepoch:
            batch += 1
            
            img = img.to(torch.float32).to(device)
            mask = mask[:, 0:1, :, :]
            mask = mask.to(torch.float32)
#             print(img.shape)
#             print(mask.shape)
            y_pred = model(img)
#             print('done')
            loss = criterion(y_pred.flatten() , mask.to(device).flatten())

            with torch.no_grad():
                train_batch_loss_list.append(loss.item())  # Convert loss to Python float
                train_epoch_loss += loss.item()
                tepoch.set_postfix(Batch_Loss=loss.item(), Batch_no=batch)

            
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

        # Log the average loss for the epoch at the end of the epoch
        with torch.no_grad():
            avg_loss = train_epoch_loss / len(train_loader)
            print(f"Epoch {epoch+1}/{num_epochs} completed. Avg Train Loss: {avg_loss:.4f}")
            train_epoch_loss_list.append(avg_loss)
    
#     with torch.no_grad():
#         predict_image(model, img, device)
    batch = 0        
    with tqdm(val_loader, unit="batch") as tepoch:
        tepoch.set_description(f"Epoch {epoch+1}/{num_epochs}")
        model.eval()
        with torch.no_grad():
            for img, mask in tepoch:
                batch += 1
            
                img = img.to(torch.float32).to(device)
                mask = mask.float()

                y_pred = model(img)
                mask = mask[:, 0:1, :, :]
#                 print(y_pred[0],mask)
                loss = criterion(y_pred, mask.to(device))

            with torch.no_grad():
                val_batch_loss_list.append(loss.item())  # Convert loss to Python float
                val_epoch_loss += loss.item()
                tepoch.set_postfix(Batch_Loss=loss.item(), Batch_no=batch)
            
            with torch.no_grad():
                avg_loss = val_epoch_loss / len(val_loader)
                print(f"Epoch {epoch+1}/{num_epochs} completed. Avg Validation Loss: {avg_loss:.4f}")
                val_epoch_loss_list.append(avg_loss)
                

            
            
    with torch.no_grad():
        Avg_IOU_Score = 0
        for img, mask in train_loader:
        
          
            img = img.float().to(device)
            mask = mask[:, 0:1, :, :]
            mask = mask.float().to('cpu').numpy()

            y_pred = model(img)
            y_pred = torch.nn.functional.sigmoid(y_pred).to('cpu').numpy()
            y_pred_binary = np.zeros(y_pred.shape)
            y_pred_binary[y_pred>=0.5] = 1
        
            IOU_Score = IOU(mask , y_pred_binary)
            Avg_IOU_Score += IOU_Score
        print(f'Avg_IOU_Score :: {Avg_IOU_Score/len(train_loader)}') 
        IOU_Score_list.append(Avg_IOU_Score/len(train_loader))
        

    with torch.no_grad():    
        # Save the model after each epoch
        model_save_name = f"{save_path}last_layer+aug{epoch+1}.pth"
        torch.save({
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': avg_loss
        }, model_save_name)

        print(f"Model saved as {model_save_name}")
        print("***************************************************************************")

In [ ]:
import torch
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt

# Function to load an image and get the model prediction
def predict_image(model, image_path, device):
    # Load the image
    image = Image.open(image_path).convert("RGB")  # Ensure it's in RGB format
    
    # Define the necessary transformations
    preprocess = transforms.Compose([
        transforms.Resize((512, 512)),  # Resize to match model input size
        transforms.ToTensor(),  # Convert image to tensor
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    # Preprocess the image
    input_image = preprocess(image).unsqueeze(0)  # Add batch dimension

    # Move the image to the specified device
    input_image = input_image.to(device)

    # Make the prediction
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        output = model(input_image)

    # If the output is a single channel (e.g., for segmentation), apply sigmoid
    if output.shape[1] == 1:  # Assuming the model outputs a single channel
        output = torch.sigmoid(output)  # Convert logits to probabilities
        output = (output > 0.5).float()  # Binarize the output

    # Squeeze the output to remove batch and channel dimensions
    output_image = output.squeeze().cpu().numpy()  # Convert to NumPy array

    # Plot the original image and the prediction
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.title("Original Image")
    plt.imshow(image)
    plt.axis("off")

    plt.subplot(1, 2, 2)
    plt.title("Model Prediction")
    plt.imshow(output_image, cmap='gray')  # Change cmap based on output
    plt.axis("off")

    plt.show()

# Example usage
image_path = '/kaggle/input/test-data/test_image.jpg'
predict_image(model, image_path, device)


In [ ]:
import cv2
import numpy as np
import torch
from torchvision import transforms

# Load the video
input_video_path = '/kaggle/input/test-video-3/test_video.mp4'
output_video_path = '/kaggle/working/output_video.mp4'

cap = cv2.VideoCapture(input_video_path)

# Define video codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for .mp4
fps = cap.get(cv2.CAP_PROP_FPS)
frame_width = 512  # Resize width
frame_height = 512  # Resize height
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# Define a transform for PyTorch model
torch_transform = transforms.Compose([
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Only normalization step
])

model.eval()  # Set to evaluation mode

frame_count = 0  # Track the number of processed frames

# Loop through video frames
while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        print("Error: Could not read frame.")
        break

    # Resize frame for model input to 512x512
    frame_resized = cv2.resize(frame, (frame_width, frame_height))

    # Convert to NumPy array and prepare for PyTorch model
    frame_resized_np = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2RGB)
    frame_resized_torch = torch.from_numpy(frame_resized_np).permute(2, 0, 1).float() / 255.0
    frame_resized_torch = torch_transform(frame_resized_torch)

    # Add batch dimension and predict the mask
    with torch.no_grad():
        mask = model(frame_resized_torch.unsqueeze(0).cuda())[0]  # Forward pass

    # Squeeze the mask to remove the batch dimension
    mask = mask.squeeze(0).cpu().numpy()  # Now it should be (512, 512)

    # Check the shape of the mask after squeezing
    if len(mask.shape) != 2:
        print(f"Error: Expected single-channel mask, but got shape: {mask.shape}")
        break

    # Apply threshold
    mask = np.where(mask > 0.8, 1, 0).astype(np.uint8)  # Assuming the mask is in [0, 1]

    # Resize mask to ensure it matches frame dimensions
    mask_resized = cv2.resize(mask, (frame_width, frame_height))

    # Create an all-red overlay where mask_resized is 1
    mask_red = np.zeros_like(frame_resized)  # Create a blank image with the same dimensions as the frame
    mask_red[:, :, 2] = mask_resized * 255  # Set red channel to 255 where mask is 1

    # Overlay red mask on the original frame
    overlayed_frame = cv2.addWeighted(frame_resized, 1, mask_red, 0.5, 0)

    # Write the processed frame to the output video
    out.write(overlayed_frame)

    frame_count += 1  # Increment the frame count
    if frame_count % 200 == 0:
        print(f"Processed frame: {frame_count}")  # Print progress

# Release the video capture and writer objects
cap.release()
out.release()

# Close all OpenCV windows (commenting out as this may cause issues in your environment)
# cv2.destroyAllWindows()

print(f"Video processing completed. Output saved at {output_video_path}")


In [ ]:
import cv2
import numpy as np
import torch
from torchvision import transforms

# Input and output paths
input_video_path = '/kaggle/working/output_video.mp4'
output_video_path = '/kaggle/working/output_video_erosion.mp4'

# Video properties
cap = cv2.VideoCapture(input_video_path)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps = cap.get(cv2.CAP_PROP_FPS)
frame_width = 512
frame_height = 512
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# PyTorch transforms
torch_transform = transforms.Compose([
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Set model to evaluation mode
model.eval()

# Parameters for erosion and filtering
min_area = 500  # Minimum area to keep regions
kernel_size = 5  # Erosion kernel size

# Function to apply region filtering based on area
def filter_regions_by_area(binary_mask, min_area):
    # Find contours in the binary mask
    contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    filtered_mask = np.zeros_like(binary_mask)

    for contour in contours:
        if cv2.contourArea(contour) > min_area:
            cv2.drawContours(filtered_mask, [contour], -1, 255, thickness=cv2.FILLED)
    return filtered_mask

# Loop through video frames
frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("End of video or error reading frame.")
        break

    # Resize frame
    resized_frame = cv2.resize(frame, (frame_width, frame_height))

    # Convert frame to RGB and tensor for model input
    rgb_frame = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2RGB)
    frame_tensor = torch.from_numpy(rgb_frame).permute(2, 0, 1).float() / 255.0
    frame_tensor = torch_transform(frame_tensor).unsqueeze(0).cuda()

    # Model prediction
    with torch.no_grad():
        mask = model(frame_tensor)[0].squeeze(0).cpu().numpy()  # (512, 512)

    # Threshold and resize mask
    binary_mask = (mask > 0.8).astype(np.uint8)  # Binary segmentation mask
    binary_mask_resized = cv2.resize(binary_mask, (frame_width, frame_height))

    # Erosion for noise removal
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernel_size, kernel_size))
    eroded_mask = cv2.erode(binary_mask_resized, kernel, iterations=1)

    # Region filtering
    filtered_mask = filter_regions_by_area(eroded_mask, min_area)

    # Depth-based or contextual filtering (optional, placeholder for depth/context constraints)
    # depth_filtered_mask = apply_depth_constraints(filtered_mask, depth_map)  # Integrate if depth is available

    # Highlight the segmented road in red on the original frame
    overlay = np.zeros_like(resized_frame)
    overlay[:, :, 2] = filtered_mask * 255  # Red channel
    overlayed_frame = cv2.addWeighted(resized_frame, 1, overlay, 0.5, 0)

    # Write the frame
    out.write(overlayed_frame)

    frame_count += 1
    if frame_count % 100 == 0:
        print(f"Processed {frame_count} frames.")

# Release resources
cap.release()
out.release()
print(f"Processed video saved at {output_video_path}")
